In [1]:
import numpy as np 
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

*Acquiring an API Token*

In [6]:
from secret import IEX_CLOUD_API_TOKEN

making an api call

In [13]:
symbol = "AAPL"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url

'https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448'

In [17]:
data = requests.get(api_url).json()
print(data['symbol'])

AAPL


In [19]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/1000000000000)

2.017625600407


adding our stocks data to a pandas dataframe

In [21]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalisation', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy


In [24]:
final_dataframe.append(
    pd.Series(
        [
            symbol, 
            price,
            market_cap,
            'N/A'
        ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,AAPL,125.414,2017625600407,N/A


In [30]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock, 
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index= my_columns
        ),
        ignore_index=True
    )

In [31]:
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,126.96,39711128125,N/A
1,AAL,24.45,15842724574,N/A
2,AAP,188.46,12362686579,N/A
3,AAPL,120.49,2099506328081,N/A
4,ABBV,110.20,195415124373,N/A
...,...,...,...,...
500,YUM,114.67,34785368857,N/A
501,ZBH,164.11,34057764632,N/A
502,ZBRA,476.71,25791672075,N/A
503,ZION,56.98,9090320267,N/A


Batch API Call

In [34]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [45]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index = True
        )
    
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,127.600,39795931133,N/A
1,AAL,24.730,15782362018,N/A
2,AAP,189.820,12425149917,N/A
3,AAPL,122.460,2017150221157,N/A
4,ABBV,111.250,195310630613,N/A
...,...,...,...,...
500,YUM,110.440,33613786569,N/A
501,ZBH,166.210,34307958353,N/A
502,ZBRA,474.563,26367444858,N/A
503,ZION,55.500,9419864944,N/A


calculating number of shares to buy

In [57]:
portfolio_size = input('Enter your portfolio value:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number \nPlease try again: ")
    portfolio_size = input('Enter your portfolio value:')
    val = float(portfolio_size)

10000000.0


In [66]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])

final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,127.600,39795931133,155
1,AAL,24.730,15782362018,800
2,AAP,189.820,12425149917,104
3,AAPL,122.460,2017150221157,161
4,ABBV,111.250,195310630613,177
...,...,...,...,...
500,YUM,110.440,33613786569,179
501,ZBH,166.210,34307958353,119
502,ZBRA,474.563,26367444858,41
503,ZION,55.500,9419864944,356


formatting our excel output

In [67]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [68]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [69]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [70]:
writer.save()